In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/home/mcamara/taxi-demand-predictor/')

In [3]:
import pandas as pd
from src.paths import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR / 'tabular_data.parquet')
df

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,target_rides_next_hour
0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,...,2.0,0.0,1.0,0.0,0.0,0.0,0.0,2022-01-29,1,0.0
1,0.0,0.0,0.0,0.0,0.0,4.0,1.0,2.0,1.0,2.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2022-01-30,1,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,1.0,2.0,0.0,0.0,0.0,0.0,2022-01-31,1,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,2.0,1.0,0.0,1.0,1.0,0.0,0.0,2022-02-01,1,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-02-02,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81085,6.0,7.0,7.0,4.0,9.0,1.0,3.0,5.0,4.0,7.0,...,5.0,8.0,7.0,2.0,1.0,5.0,1.0,2022-11-26,265,2.0
81086,10.0,5.0,4.0,11.0,8.0,2.0,2.0,5.0,7.0,8.0,...,6.0,2.0,1.0,3.0,2.0,8.0,4.0,2022-11-27,265,3.0
81087,6.0,2.0,2.0,0.0,3.0,7.0,24.0,20.0,10.0,18.0,...,5.0,5.0,4.0,4.0,8.0,1.0,7.0,2022-11-28,265,10.0
81088,4.0,6.0,0.0,1.0,1.0,7.0,22.0,18.0,25.0,17.0,...,10.0,11.0,13.0,3.0,4.0,3.0,1.0,2022-11-29,265,3.0


In [4]:
from datetime import datetime
from src.data_split import train_test_split

X_train, y_train, X_test, y_test = train_test_split(
    df,
    cutoff_date=datetime(2022, 6, 1, 0, 0, 0),
    target_column_name='target_rides_next_hour'
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

X_train.shape=(32595, 674)
y_train.shape=(32595,)
X_test.shape=(48495, 674)
y_test.shape=(48495,)


In [5]:
import catboost as ctb

In [6]:
past_rides_columns = [c for c in X_train.columns if c.startswith('rides_')]
X_train_only_numeric = X_train[past_rides_columns]

In [7]:
model = ctb.CatBoostRegressor()
model.fit(X_train_only_numeric, y_train)

Learning rate set to 0.071
0:	learn: 33.8520012	total: 763ms	remaining: 12m 41s
1:	learn: 31.7814159	total: 876ms	remaining: 7m 17s
2:	learn: 29.8772843	total: 970ms	remaining: 5m 22s
3:	learn: 28.1561642	total: 1.09s	remaining: 4m 31s
4:	learn: 26.5453620	total: 1.19s	remaining: 3m 57s
5:	learn: 25.0112418	total: 1.32s	remaining: 3m 39s
6:	learn: 23.6250008	total: 1.43s	remaining: 3m 22s
7:	learn: 22.3101887	total: 1.56s	remaining: 3m 14s
8:	learn: 21.0731546	total: 1.71s	remaining: 3m 8s
9:	learn: 19.9106740	total: 1.81s	remaining: 2m 59s
10:	learn: 18.8709658	total: 1.92s	remaining: 2m 52s
11:	learn: 17.8599642	total: 2.04s	remaining: 2m 48s
12:	learn: 16.9581693	total: 2.19s	remaining: 2m 46s
13:	learn: 16.1151185	total: 2.31s	remaining: 2m 42s
14:	learn: 15.3558924	total: 2.41s	remaining: 2m 38s
15:	learn: 14.6429872	total: 2.55s	remaining: 2m 37s
16:	learn: 14.0044286	total: 2.7s	remaining: 2m 36s
17:	learn: 13.4139440	total: 2.87s	remaining: 2m 36s
18:	learn: 12.8401067	total: 3

In [8]:
X_test_only_numeric = X_test[past_rides_columns]
predictions = model.predict(X_test_only_numeric)

from sklearn.metrics import mean_absolute_error
test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:.4f}')

test_mae=2.4723
